In [ ]:
!pip install torchsummaryX
!pip install -q cohere tiktoken openai==0.28
!pip install -q datasets
!pip install -q wandb

In [ ]:
!pip install evaluate -q
!pip install rouge_score -q
!pip install nltk -q
!pip install absl -q

ERROR: Could not find a version that satisfies the requirement absl (from versions: none)
ERROR: No matching distribution found for absl


In [ ]:
import gc
from torchsummaryX import summary
import torch
import torch.nn as nn
import tiktoken
from datasets import load_dataset
import wandb
from tqdm import tqdm
import numpy as np
import os
import math

In [ ]:
TRAIN = True

# block_size = 100
block_size = 800
# batch_size = 128
batch_size = 20
num_embeddings = 384
dropout = 0.2
num_heads = 6
num_layers = 6
# percent_use = 0.5
epochs = 32
lr = 1e-3
# vocab_size = tiktoken.get_encoding("r50k_base").n_vocab
vocab_size = 100265
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Device: ", device)

Device:  cuda


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
config = {
    'root': '/content/drive/MyDrive/11685-hws/hw5/data',
    'batch_size': batch_size,
    'block_size': block_size,
    'num_epochs': epochs
}

In [ ]:
# gc.collect()
# torch.cuda.empty_cache()

# pre-train data
# train_data = np.memmap(os.path.join(config['root'], 'train.bin'), dtype=np.uint16, mode='r')
# val_data = np.memmap(os.path.join(config['root'], 'val.bin'), dtype=np.uint16, mode='r')
# # print(train_data.shape)

In [ ]:
# torch.manual_seed(1337)
# def get_batch(split, batch_idx):
#     data = train_data if split == 'train' else val_data
#     idx = batch_idx * config['block_size'] * config['batch_size']
#     # ix = torch.randint(len(data) - config['block_size'], (config['batch_size'],))
#     # x = torch.from_numpy(data[idx:idx+config['block_size']].astype(np.int64))
#     # y = torch.from_numpy(data[idx+1:idx+1+config['block_size']].astype(np.int64))
#     x = torch.stack([torch.from_numpy((data[i:i+config['block_size']]).astype(np.int64)) for i in range(idx, idx + config['batch_size'] * config['block_size'], config['block_size'])])
#     y = torch.stack([torch.from_numpy((data[i+1:i+1+config['block_size']]).astype(np.int64)) for i in range(idx, idx + config['batch_size'] * config['block_size'], config['block_size'])])
#     x, y = x.pin_memory().to(device, non_blocking=True), y.pin_memory().to(device, non_blocking=True)
#     return x, y

# print("Batch size: ", config['batch_size'])
# train_data_len = len(train_data)
# train_data_batches = train_data_len // (config['batch_size'] * config['block_size'])
# train_data_batches = train_data_batches // 32
# val_data_len = len(val_data)
# val_data_batches = val_data_len // (config['batch_size'] * config['block_size'])
# val_data_batches = val_data_batches // 8
# print("Train dataset samples = {}, batches = {}".format(train_data_len, train_data_batches))
# print("Val dataset samples = {}, batches = {}".format(val_data_len, val_data_batches))


# xb, yb = get_batch('train', 0)
# print('inputs:')
# print(xb.shape)
# print(xb)
# print('targets:')
# print(yb.shape)
# print(yb)

# print('----')

# xb, yb = get_batch('train', 1)
# print('inputs:')
# print(xb.shape)
# print(xb)
# print('targets:')
# print(yb.shape)
# print(yb)

In [ ]:
# load SQuAD data
import json

def read_squad(path):
    # open JSON file and load intro dictionary
    with open(path, 'rb') as f:
        squad_dict = json.load(f)

    # initialize lists for contexts, questions, and answers
    contexts = []
    questions = []
    answers = []
    # iterate through all data in squad data
    for group in squad_dict['data']:
        for passage in group['paragraphs']:
            context = passage['context']
            for qa in passage['qas']:
                question = qa['question']
                # check if we need to be extracting from 'answers' or 'plausible_answers'
                if 'plausible_answers' in qa.keys():
                    access = 'plausible_answers'
                else:
                    access = 'answers'
                for answer in qa[access]:
                    # append data to lists
                    contexts.append(context)
                    questions.append(question)
                    answers.append(answer)
    # return formatted data lists
    return contexts, questions, answers

# execute our read SQuAD function for training and validation sets
train_contexts, train_questions, train_answers = read_squad(os.path.join(config['root'], 'squad/train-v2.0.json'))
val_contexts, val_questions, val_answers = read_squad(os.path.join(config['root'], 'squad/dev-v2.0.json'))

In [ ]:
print(train_contexts[0])
print(train_questions[0])
print(train_answers[0])

Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny's Child. Managed by her father, Mathew Knowles, the group became one of the world's best-selling girl groups of all time. Their hiatus saw the release of Beyoncé's debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles "Crazy in Love" and "Baby Boy".
When did Beyonce start becoming popular?
{'text': 'in the late 1990s', 'answer_start': 269}


In [ ]:
train_answers_text = [answer['text'] for answer in train_answers]
val_answers_text = [answer['text'] for answer in val_answers]
print(train_answers_text[0])

in the late 1990s


In [ ]:
# load encoder
cl100k_base = tiktoken.get_encoding("cl100k_base")

# In production, load the arguments directly instead of accessing private attributes
# See openai_public.py for examples of arguments for specific encodings
enc = tiktoken.Encoding(
    # If you're changing the set of special tokens, make sure to use a different name
    # It should be clear from the name what behaviour to expect.
    name="cl100k_im",
    pat_str=cl100k_base._pat_str,
    mergeable_ranks=cl100k_base._mergeable_ranks,
    special_tokens={
        **cl100k_base._special_tokens,
        "<|pad|>": 100264,
    }
)
pad_value = enc.encode("<|pad|>", allowed_special="all")[0]

In [ ]:
answer_prompt = enc.encode('\nanswer: ')
import torch
import numpy as np

class SquadDataset(torch.utils.data.Dataset):
    def __init__(self, context, question, answer, encoder, block_size):
        self.block_size = block_size
        # add 'context: ' to the beginning of each context so that our model knows where the context begins
        context = ['context: ' + text for text in context]
        # add 'question: ' to the beginning of each answer so that our model knows where the answer begins
        question = ['question: ' + text for text in question]
        # add 'answer: ' to the beginning of each answer so that our model knows where the answer begins
        # answer = ['answer: ' + text for text in answer]
        answer = [text for text in answer]

        # concat our context and question into one string and separate them with a new line
        # trucate our context/question to the maximum length the model can handle
        # note: this is a special token used by GPT2 to indicate the end of text
        self.X = []
        for context, question in zip(context, question):
            context = encoder.encode(context + ' \n')
            question = encoder.encode(question)
            if len(context) + len(question) > block_size - len(answer_prompt):
                context = context[:block_size - len(answer_prompt) - len(question)]
            context.extend(question)
            if len(context) > block_size - len(answer_prompt):
                context = context[:block_size - len(answer_prompt)]
            self.X.append(context)
        self.Y = []
        for text in answer:
            text = encoder.encode(text)
            if len(text) > block_size:
                text = text[:block_size]
            self.Y.append(text)
        # self.X = [context + '\n' + question + '\nanswer: ' for context, question in zip(context, question)]
        # tokenize our context, question and answer strings
        # self.X = encoder.encode_batch(self.X)
        # self.Y = encoder.encode_batch(answer)
        # self.X = np.array(self.X)
        # self.Y = np.array(self.Y)
        assert len(self.X) == len(self.Y)

    def __getitem__(self, idx):
        x = self.X[idx]
        y = self.Y[idx]
        return torch.tensor(x), torch.tensor(y)

    def __len__(self):
        return len(self.X)//4

    def collate_fn(self,batch):

        batch_x, batch_y, lengths_x, lengths_y = [], [], [], []

        for x, y in batch:
            # Add the mfcc, transcripts and their lengths to the lists created above
            # pad x and y with zeros to self.block_size
            lengths_x.append(x.shape[0])
            lengths_y.append(y.shape[0])
            try:
                x = np.pad(x, (0, self.block_size - len(answer_prompt) - len(x)), 'constant', constant_values=pad_value)
                y = np.pad(y, (0, self.block_size - len(y)), 'constant', constant_values=pad_value)
            except:
                print(x.shape, y.shape)
                print(x, y)
                raise
            x = np.concatenate((x, answer_prompt))
            batch_x.append(x)
            batch_y.append(y)

        batch_x = np.array(batch_x)
        batch_y = np.array(batch_y)
        # pack the mfccs and transcripts using the pad_sequence function from pytorch
        # batch_x_pad = torch.nn.utils.rnn.pad_sequence(batch_x, batch_first=True, padding_value=0)
        # batch_y_pad = torch.nn.utils.rnn.pad_sequence(batch_y, batch_first=True, padding_value=0)

        return torch.tensor(batch_x), torch.tensor(batch_y), torch.tensor(lengths_x), torch.tensor(lengths_y)

# build datasets for both our training and validation sets
train_dataset_squad = SquadDataset(train_contexts, train_questions, train_answers_text, enc, config['block_size'])
val_dataset_squad = SquadDataset(val_contexts, val_questions, val_answers_text, enc, config['block_size'])

In [ ]:
# load CNN Dailymail dataset
dataset = load_dataset('cnn_dailymail', '3.0.0')

In [ ]:
train_article_total = [x['article'] for x in dataset['train']]
train_highlights_total = [x['highlights'] for x in dataset['train']]
val_article_total = [x['article'] for x in dataset['validation']]
val_highlights_total = [x['highlights'] for x in dataset['validation']]

In [ ]:
idx = 2
fraction = 8
train_article = train_article_total[idx*len(train_article_total)//fraction:(idx+1)*len(train_article_total)//fraction]
train_highlights = train_highlights_total[idx*len(train_highlights_total)//fraction:(idx+1)*len(train_highlights_total)//fraction]
val_article = val_article_total[:len(val_article_total)//8]
val_highlights = val_highlights_total[:len(val_highlights_total)//8]

In [ ]:
summary_prompt = enc.encode('\nSummary: ')
class CNNDailyDataset(torch.utils.data.Dataset):
    def __init__(self, article, highlights, encoder, block_size):
        self.block_size = block_size
        self.X = []
        for text in article:
            text = 'Summarize this article: ' + text
            text = encoder.encode(text[:block_size])
            # if len(text) > block_size:
            #     text = text[:block_size]
            self.X.append(text)

        self.Y = []
        for text in highlights:
            # text = 'Summary: ' + text
            text = encoder.encode(text[:block_size])
            # if len(text) > block_size:
            #     text = text[:block_size]
            self.Y.append(text)
        assert len(self.X) == len(self.Y)

    def __getitem__(self, idx):
        x = self.X[idx]
        y = self.Y[idx]
        return torch.tensor(x), torch.tensor(y)

    def __len__(self):
        return len(self.X)

    def collate_fn(self,batch):

        batch_x, batch_y, lengths_x, lengths_y = [], [], [], []

        for x, y in batch:
            # Add the mfcc, transcripts and their lengths to the lists created above
            # pad x and y with zeros to self.block_size
            lengths_x.append(x.shape[0])
            lengths_y.append(y.shape[0])
            x = np.pad(x, (0, self.block_size - len(summary_prompt) - len(x)), 'constant', constant_values=pad_value)
            y = np.pad(y, (0, self.block_size - len(y)), 'constant', constant_values=pad_value)
            x = np.concatenate((x, summary_prompt))
            # y = np.concatenate((y, answer_prompt))
            batch_x.append(x)
            batch_y.append(y)

        batch_x = np.array(batch_x)
        batch_y = np.array(batch_y)
        # pack the mfccs and transcripts using the pad_sequence function from pytorch
        # batch_x_pad = torch.nn.utils.rnn.pad_sequence(batch_x, batch_first=True, padding_value=0)
        # batch_y_pad = torch.nn.utils.rnn.pad_sequence(batch_y, batch_first=True, padding_value=0)

        return torch.tensor(batch_x), torch.tensor(batch_y), torch.tensor(lengths_x), torch.tensor(lengths_y)

# build datasets for both our training and validation sets
train_dataset_cnn = CNNDailyDataset(train_article, train_highlights, enc, config['block_size'])
val_dataset_cnn = CNNDailyDataset(val_article, val_highlights, enc, config['block_size'])

In [ ]:
train_loader_cnn    = torch.utils.data.DataLoader(
    dataset     = train_dataset_cnn,
    batch_size  = config['batch_size'],
    shuffle     = True,
    num_workers = 4,
    pin_memory  = True,
    drop_last   = True,
    collate_fn  = train_dataset_cnn.collate_fn
)

valid_loader_cnn    = torch.utils.data.DataLoader(
    dataset     = val_dataset_cnn,
    batch_size  = config['batch_size'],
    shuffle     = False,
    num_workers = 2,
    pin_memory  = True,
    drop_last   = True,
    collate_fn  = val_dataset_cnn.collate_fn
)

print("No. of train mfccs   : ", train_dataset_cnn.__len__())
print("Batch size           : ", config['batch_size'])
print("Train batches        : ", train_loader_cnn.__len__())
print("Valid batches        : ", valid_loader_cnn.__len__())

print("\nChecking the shapes of the data...")
for batch in train_loader_cnn:
    x, y, x_len, y_len = batch
    print(x.shape, y.shape, x_len.shape, y_len.shape)
    print(x[0])
    print(enc.decode(x[0].tolist()))
    print(y[0])
    print(enc.decode(y[0].tolist()))
    print(y_len[0])
    break

No. of train mfccs   :  35889
Batch size           :  20
Train batches        :  1794
Valid batches        :  83

Checking the shapes of the data...
torch.Size([20, 800]) torch.Size([20, 800]) torch.Size([20]) torch.Size([20])
tensor([  9370,   5730,    553,    420,   4652,     25,   3296,    662,   7957,
         58344,     11,   5492,  23298,  12865,    662,    393,  14451,  39979,
            25,    662,    220,    777,     25,   2371,  26963,     11,    220,
          1627,   6841,    220,    679,     17,    662,    765,    662,  78961,
            25,    662,    220,   2437,     25,   1987,  26963,     11,    220,
          1544,   6841,    220,    679,     17,    662,    326,   6564,  20550,
          2011,    387,   2728,   1949,   6514,    555,    480,  21051,     11,
          1234,    264,  49737,  20733,   1608,    858,  23415,   1523,    555,
         37381,  20258,     13,    578,  16410,    690,    387,   7318,   7083,
          1227,    264,   1060,    369,   1855,  5221

In [ ]:
train_loader_squad    = torch.utils.data.DataLoader(
    dataset     = train_dataset_squad,
    batch_size  = config['batch_size'],
    shuffle     = True,
    num_workers = 4,
    pin_memory  = True,
    drop_last   = True,
    collate_fn  = train_dataset_squad.collate_fn
)

valid_loader_squad    = torch.utils.data.DataLoader(
    dataset     = val_dataset_squad,
    batch_size  = config['batch_size'],
    shuffle     = False,
    num_workers = 2,
    pin_memory  = True,
    drop_last   = True,
    collate_fn  = val_dataset_squad.collate_fn
)

print("No. of train mfccs   : ", train_dataset_squad.__len__())
print("Batch size           : ", config['batch_size'])
print("Train batches        : ", train_loader_squad.__len__())
print("Valid batches        : ", valid_loader_squad.__len__())

print("\nChecking the shapes of the data...")
for batch in train_loader_squad:
    x, y, x_len, y_len = batch
    print(x.shape, y.shape, x_len.shape, y_len.shape)
    print(x[0])
    print(enc.decode(x[0].tolist()))
    print(y[0])
    print(enc.decode(y[0].tolist()))
    print(y_len[0])
    break

No. of train mfccs   :  32579
Batch size           :  20
Train batches        :  1628
Valid batches        :  327

Checking the shapes of the data...
torch.Size([20, 800]) torch.Size([20, 800]) torch.Size([20]) torch.Size([20])
tensor([  2196,     25,  35132,   6072,    369,  22735,   5045,    527,  20816,
           304,   3116,  21562,     25,    469,     16,     11,    469,     17,
            11,    469,     20,     33,     11,    323,    469,     21,    323,
         28347,    449,  10845,  85234,     13,    578,   2144,    315,  50917,
          1436,    387,  17125,    505,    279,   1486,    315,   1684,    315,
           279,  14291,   2955,     11,    719,    389,    279,   1023,   1450,
         25930,    279,   4819,    315,    958,  37748,  32317,     11,   5423,
          2949,    469,     16,    323,    469,     17,  21562,     11,    902,
           527,  20816,    369,  10845,  85234,    596,  17880,  35319,   2532,
            13,   4452,     11,   1234,   7327,  138

In [ ]:
# model
class self_attention(nn.Module):
    def __init__(self, head_size, num_embeddings):
        super().__init__()
        self.query = nn.Linear(num_embeddings, head_size, bias=False)
        self.key = nn.Linear(num_embeddings, head_size, bias=False)
        self.value = nn.Linear(num_embeddings, head_size, bias=False)
        self.softmax = nn.Softmax(dim=-1)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        self.Q = self.query(x)
        self.K = self.key(x)
        self.V = self.value(x)

        t = self.Q.size(dim=1)
        lower_t = torch.tril(torch.ones(t, t)).to(device)
        self.A_w = torch.matmul(self.Q, torch.permute(self.K, (0, 2, 1)))
        d_k_root = torch.sqrt(torch.tensor(self.K.size(dim=2)))
        self.A_w = self.A_w.masked_fill(lower_t == 0, float('-inf'))
        self.A_sig = self.softmax(torch.div(self.A_w, d_k_root))
        self.A_sig = self.dropout(self.A_sig)
        X_new = torch.bmm(self.A_sig, self.V)

        return X_new

class MultiheadAttention(nn.Module):

    def __init__(self, num_heads, head_size, num_embeddings):
        super(MultiheadAttention, self).__init__()
        self.multihead = nn.ModuleList([self_attention(head_size, num_embeddings) for _ in range(num_heads)])
        self.proj = nn.Linear(num_embeddings, num_embeddings)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([head(x) for head in self.multihead], dim=-1)
        return self.dropout(self.proj(out))

class FeedForward(nn.Module):

    def __init__(self):
        super(FeedForward, self).__init__()
        self.block = nn.Sequential(nn.Linear(num_embeddings, 4 * num_embeddings), nn.GELU(),
                                   nn.Linear(4 * num_embeddings, num_embeddings), nn.Dropout(dropout))

    def forward(self, x):
        return self.block(x)

class TransformerBlock(nn.Module):

    def __init__(self):
        super(TransformerBlock, self).__init__()
        self.attention = MultiheadAttention(num_heads=num_heads, head_size=num_embeddings // num_heads,
                                            num_embeddings=num_embeddings)
        self.feed_forward = FeedForward()
        self.ln1, self.ln2 = nn.LayerNorm(num_embeddings), nn.LayerNorm(num_embeddings)

    def forward(self, x):
        x = x + self.attention(self.ln1(x))
        x = x + self.feed_forward(self.ln2(x))
        return x

class Model(nn.Module):

    def __init__(self):
        super().__init__()
        self.tok_embedding = nn.Embedding(vocab_size, num_embeddings)
        self.pos_embedding = nn.Embedding(block_size, num_embeddings)
        self.transformer_blocks = nn.Sequential(*[TransformerBlock() for _ in range(num_layers)])
        self.final_ln = nn.LayerNorm(num_embeddings)
        self.final_linear = nn.Linear(num_embeddings, vocab_size)
        self.B = 0
        self.T = 0
        self.C = 0
        # self.tok_embedding.weight = self.final_linear.weight
    def forward(self, x):
        tok_embed = self.tok_embedding(x)
        pos_embed = self.pos_embedding(torch.arange(block_size, device=device))
        x = tok_embed + pos_embed
        x = self.transformer_blocks(x)
        x = self.final_ln(x)
        logits = self.final_linear(x)
        B, T, C = logits.shape
        self.B = B
        self.T = T
        self.C = C
        logits = logits.view(B * T, C)

        return logits

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        _,T = idx.shape
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits = self(idx_cond)
            logits = logits.view(self.B, self.T, self.C)
            # focus only on the last time step
            logits = logits[:, -1, :]  # becomes (B, C)
            # apply softmax to get probabilities
            probs = nn.functional.softmax(logits, dim=-1)  # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1)  # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1)  # (B, T+1)
        return idx[:,T:]

In [ ]:
gc.collect()
torch.cuda.empty_cache()
model = Model().to(device)
# summary(model, xb.to(device))
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
criterion = torch.nn.CrossEntropyLoss()
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=2)
scaler = torch.cuda.amp.GradScaler()

In [ ]:
# Pre-train
# from tqdm import tqdm

# def train_model(model, criterion, optimizer, start):

#     model.train()
#     batch_bar = tqdm(total=train_data_batches, dynamic_ncols=True, leave=False, position=0, desc='Train')

#     total_loss = 0
#     minibatch = 0
#     for k in range(start, start + train_data_batches):
#         # print(k)
#         # try:
#         X, Y = get_batch('train', k)
#         with torch.cuda.amp.autocast():
#             logits = model(X)
#             loss = criterion(logits.view(-1, logits.size(-1)), Y.view(-1))
#         total_loss += loss.item()
#         batch_bar.set_postfix(
#             loss="{:.04f}".format(float(total_loss / (k + 1))),
#             lr="{:.06f}".format(float(optimizer.param_groups[0]['lr'])))
#         # Add this check during training loop
#         if math.isnan(total_loss):
#             print("NaN loss encountered. Stopping training.")
#             print(X)
#             print(Y)
#             print(logits)
#             print(loss)
#             break
#         batch_bar.update() # Update tqdm bar
#         # Another couple things you need for FP16.
#         scaler.scale(loss).backward() # This is a replacement for loss.backward()
#         scaler.step(optimizer) # This is a replacement for optimizer.step()
#         scaler.update() # This is something added just for FP16

#         del X, Y, loss
#         torch.cuda.empty_cache()
#         gc.collect()
#         # print("minibatch finished: ", minibatch)
#         minibatch += 1
#         # except:
#         #     print(f'{k} batch dropped')
#         optimizer.zero_grad()


#     batch_bar.close() # You need this to close the tqdm bar

#     return total_loss / train_data_batches, k


# def validate_model(model, criterion):

#     model.eval()
#     batch_bar = tqdm(total=val_data_batches, dynamic_ncols=True, position=0, leave=False, desc='Val')

#     total_loss = 0

#     for k in range(val_data_batches):

#         # try:
#         X, Y = get_batch('val', k)
#         with torch.inference_mode():
#             logits = model(X)
#             loss = criterion(logits.view(-1, logits.size(-1)), Y.view(-1))

#         total_loss += float(loss)

#         batch_bar.set_postfix(loss="{:.04f}".format(float(total_loss / (k + 1))))

#         batch_bar.update()

#         del X, Y, loss
#         torch.cuda.empty_cache()
#         gc.collect()
#         # except:
#         #     print(f'{k} batch dropped')



#     batch_bar.close()
#     total_loss = total_loss/val_data_batches

#     return total_loss

In [ ]:
def save_model(model, optimizer, scheduler, loss, epoch, path):
    torch.save(
        {'model_state_dict'         : model.state_dict(),
         'optimizer_state_dict'     : optimizer.state_dict(),
         'scheduler_state_dict'     : scheduler.state_dict(),
         'epoch'                    : epoch,
         'loss'                     : loss,
         },
         path
    )

def load_model(path, model, optimizer= None, scheduler= None):

    checkpoint = torch.load(path)
    model.load_state_dict(checkpoint['model_state_dict'])

    if optimizer != None:
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    if scheduler != None:
        scheduler.load_state_dict(checkpoint['scheduler_state_dict'])

    epoch   = checkpoint['epoch']
    loss  = checkpoint['loss']

    return [model, optimizer, scheduler, epoch, loss]

In [ ]:
def itr_merge(*itrs):
    for itr in itrs:
        for v in itr:
            yield v

In [ ]:
# Fine-tune
from tqdm import tqdm

def train_model(model, criterion, optimizer):

    model.train()
    batch_bar = tqdm(total=len(train_loader_squad)+len(train_loader_cnn), dynamic_ncols=True, leave=False, position=0, desc='Train')

    total_loss = 0
    minibatch = 0
    # for i, data in enumerate(train_loader):
    for i, data in enumerate(itr_merge(train_loader_cnn, train_loader_squad)):
        # print(k)
        # try:
        X, Y = data[0].to(device), data[1].to(device)
        with torch.cuda.amp.autocast():
            logits = model(X)
            loss = criterion(logits.view(-1, logits.size(-1)), Y.view(-1))
        total_loss += loss.item()
        batch_bar.set_postfix(
            loss="{:.04f}".format(float(total_loss / (i + 1))),
            lr="{:.06f}".format(float(optimizer.param_groups[0]['lr'])))
        # Add this check during training loop
        if math.isnan(total_loss):
            print("NaN loss encountered. Stopping training.")
            print(X)
            print(Y)
            print(logits)
            print(loss)
            break
        batch_bar.update() # Update tqdm bar
        # Another couple things you need for FP16.
        scaler.scale(loss).backward() # This is a replacement for loss.backward()
        scaler.step(optimizer) # This is a replacement for optimizer.step()
        scaler.update() # This is something added just for FP16

        del X, Y, loss
        torch.cuda.empty_cache()
        gc.collect()
        # print("minibatch finished: ", minibatch)
        minibatch += 1
        # except:
        #     print(f'{k} batch dropped')
        optimizer.zero_grad()


    batch_bar.close() # You need this to close the tqdm bar

    return total_loss / (len(train_loader_squad)+len(train_loader_cnn))


def validate_model(model, criterion):

    model.eval()
    batch_bar = tqdm(total=len(valid_loader_squad)+len(valid_loader_cnn), dynamic_ncols=True, position=0, leave=False, desc='Val')

    total_loss = 0

    # for i, data in enumerate(valid_loader):
    for i, data in enumerate(itr_merge(valid_loader_cnn, valid_loader_squad)):

        # try:
        X, Y = data[0].to(device), data[1].to(device)
        with torch.inference_mode():
            logits = model(X)
            loss = criterion(logits.view(-1, logits.size(-1)), Y.view(-1))

        total_loss += float(loss)

        batch_bar.set_postfix(loss="{:.04f}".format(float(total_loss / (i + 1))))

        batch_bar.update()

        del X, Y, loss
        torch.cuda.empty_cache()
        gc.collect()
        # except:
        #     print(f'{k} batch dropped')



    batch_bar.close()
    total_loss = total_loss/(len(valid_loader_squad)+len(valid_loader_cnn))

    return total_loss

In [ ]:
wandb.login(key="87d4ded850f739673d588bb48fc800a2590c2f12")
run = wandb.init(
    # name="LLM-Training-100k",  ## Wandb creates random run names if you skip this field
    name="LLM-Training-100k-800-finetune",  ## Wandb creates random run names if you skip this field
    # reinit=True,  ### Allows reinitalizing runs when you re-run this cell
    id = '5rphxddv',### Insert specific run id here if you want to resume a previous run
    resume = "must", ### You need this to resume previous runs, but comment out reinit = True when using this
    project="hw5-ablations",  ### Project should be created in your wandb account
)

wandb: Currently logged in as: jding3. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
best_model_path = '/content/drive/MyDrive/11685-hws/hw5/model/best_checkpoint-800-finetune.pth'
model, optimizer, scheduler, start, best_loss = load_model(best_model_path, model, optimizer, scheduler)
start += 1
print(start)
print(best_loss)

1
8.642220121476708


In [ ]:
epoch_model_path = '/content/drive/MyDrive/11685-hws/hw5/model/checkpoint-800-finetune.pth'
model, optimizer, scheduler, start, best_loss = load_model(epoch_model_path, model, optimizer, scheduler)
start += 1
print(start)
print(best_loss)

2
8.75435784153822


In [ ]:
from nltk.translate.bleu_score import sentence_bleu
# compute bleu
for i, batch in enumerate(valid_loader_cnn):
    x, y, x_len, y_len = batch
    ave_len = torch.mean(y_len.float())
    predictions = model.generate(x.to(device), int(ave_len))
    predictions = enc.decode_batch(predictions.tolist())
    references = [enc.decode(y[i].tolist()) for i in range(y.shape[0])]
    references = [reference.replace('<|pad|>', '') for reference in references]
    references = [reference.split() for reference in references]
    for prediction in predictions:
        prediction = prediction.split()
        score += sentence_bleu(references, prediction)
    if i == 10:
        break
    del x, y, x_len, y_len, predictions, references
print(score/40)

In [ ]:
# compute rouge
import evaluate
rouge = evaluate.load('rouge')
predictions = []
references = []
score = 0
for i, batch in enumerate(valid_loader_squad):
    x, y, x_len, y_len = batch
    ave_len = torch.mean(y_len.float())
    predictions = model.generate(x.to(device), int(ave_len))
    predictions = enc.decode_batch(predictions.tolist())
    references = [enc.decode(y[i].tolist()) for i in range(y.shape[0])]
    predictions.extend(predictions)
    references.extend(references)
    del x, y, x_len, y_len
    if i == 10:
        break
score = rouge.compute(predictions=predictions, references=references)
print(score)

In [ ]:
# start = 0
criterion = nn.CrossEntropyLoss(ignore_index=100264)
# start_batch = 2090 * start
# start_batch = 0
# best_loss = float('inf')
best_model_path = '/content/drive/MyDrive/11685-hws/hw5/model/best_checkpoint-800-finetune.pth'
epoch_model_path = '/content/drive/MyDrive/11685-hws/hw5/model/checkpoint-800-finetune.pth'

for epoch in range(start, config['num_epochs']):
    # gc.collect()
    # torch.cuda.empty_cache()
    # start_batch = epoch * 8163
    print("\nEpoch: {}/{}".format(epoch+1, config['num_epochs']))

    # Call train and validate, get attention weights from training
    optimizer.param_groups[0]['lr'] /= 10
    curr_lr = optimizer.param_groups[0]['lr']
    # print(start_batch)
    # train_loss, end_batch = train_model(model, criterion, optimizer, int(start_batch))
    # if epoch % 2 == 0:
    # start_batch = end_batch - train_data_batches // 2 + 1
    # valid_loss = validate_model(model, criterion)
    train_loss = train_model(model, criterion, optimizer)
    valid_loss = validate_model(model, criterion)
    # Print your metrics
    print("\tTrain Loss {:.04f}\t Learning Rate {:.07f}".format(train_loss, curr_lr))
    print("\tVal Loss {:.04f}%\t".format(valid_loss))
    # Log metrics to Wandb
    wandb.log({
        'train_loss': train_loss,
        'valid_loss': valid_loss,
        'lr'        : curr_lr,
    })

    # Optional: Scheduler Step / Teacher Force Schedule Step
    scheduler.step(valid_loss)
    # tf_rate = max(tf_rate - 0.05, 0.0)

    save_model(model, optimizer, scheduler, valid_loss, epoch, epoch_model_path)
    wandb.save(epoch_model_path)
    print("Saved epoch model")

    if valid_loss <= best_loss:
        best_loss = valid_loss
        save_model(model, optimizer, scheduler, valid_loss, epoch, best_model_path)
        wandb.save(best_model_path)
        print("Saved best model")


Epoch: 3/32


Train:  50%|█████     | 1716/3422 [27:43<27:11,  1.05it/s, loss=7.7393, lr=0.000010]

In [ ]:
# best_model_path = '/content/drive/MyDrive/11685-hws/hw5/model/best_checkpoint-800.pth'
# save_model(model, optimizer, scheduler, valid_loss, epoch, best_model_path)

In [ ]:
from nltk.translate.bleu_score import sentence_bleu
model, optimizer, scheduler, start, best_loss = load_model(epoch_model_path, model, optimizer, scheduler)
score = 0
# compute bleu
for i, batch in enumerate(valid_loader_cnn):
    x, y, x_len, y_len = batch
    ave_len = torch.mean(y_len.float())
    predictions = model.generate(x, ave_len)
    predictions = enc.decode_batch(predictions.tolist())
    references = [enc.decode(y[i].tolist()) for i in range(y.shape[0])]
    references = [reference.replace('<|pad|>', '') for reference in references]
    references = [reference.split() for reference in references]
    for prediction in predictions:
        prediction = prediction.split()
        score += sentence_bleu(reference, candidate)
    if i == 10:
        break
print(score/40)